In [1]:
import os
os.chdir('..')

import pandas as pd
import torch
from tqdm import tqdm

tqdm.pandas()

from scr.model import Model

a:\python\312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
a:\python\312\Lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model = Model()
model.eval()
model.load_state_dict(torch.load('models/model_epoch_50.pt'))

C:\Users\timof\AppData\Local\Temp\ipykernel_19588\3061579655.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/model_epoch_50.pt')

<All keys matched successfully>

In [3]:
df = pd.read_csv('data/data.csv')
df['comment'] = df['comment'].str.replace('\n', '', regex=False)
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [4]:
df['predict'] = df['comment'].progress_apply(lambda x: round(float(model.predict(x).item()), 3))
df

100%|██████████| 14412/14412 [02:38<00:00, 90.67it/s]


,comment,toxic,predict
0,"Верблюдов-то за что? Дебилы, бл...",1.0,1.000
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,1.000
2,Собаке - собачья смерть,1.0,1.000
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,1.000
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,1.000
...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,1.000
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,0.487
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,0.499
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,1.000


In [8]:
(df['toxic'] == df['predict'].apply(round).astype(int)).value_counts()

True     14056
False      356
Name: count, dtype: int64